In [1]:
import ortools
from ortools.linear_solver import pywraplp
import time
import numpy as np

In [2]:
filename = 'test_500_5000.txt'
f_i = 'Data/'+ filename
f_o = 'Data/res_'+filename

t= time.time()
f = open(f_i, "r")
data = {}
line = f.readline().split()
data['n']= int(line[0])  
data['m']= int(line[1])  

data['xh'] = np.zeros(shape=(data['n']))
data['yh'] = np.zeros(shape=(data['n']))
data['vh'] = np.zeros(shape=(data['n']))
for i in range(data['n']):
    line =f.readline().split()
    data['xh'][i]=int(line[0])
    data['yh'][i]=int(line[1])
    data['vh'][i]=int(line[2])

data['xc'] = np.zeros(shape=(data['m']))
data['yc'] = np.zeros(shape=(data['m']))
data['vc'] = np.zeros(shape=(data['m']))
for i in range(data['m']):
    line =f.readline().split()
    data['xc'][i]=int(line[0])
    data['yc'][i]=int(line[1])
    data['vc'][i]=int(line[2])

f.close()

In [3]:
def distance(hub, customer):
    return np.sqrt((data['xh'][hub]-data['xc'][customer])**2+(data['yh'][hub]-data['yc'][customer])**2)

data['nearestHub'] = []
for customer in range(data['m']):
    d = np.array([distance(hub, customer) for hub in range(data['n'])]) 
    top5=[]
    for i in range(min(5,len(d))):
        top5.append(d.argsort()[i])
    data['nearestHub'].append(top5)

In [4]:
solver = pywraplp.Solver.CreateSolver('SCIP')

# Biến
x = {}
# x[i,j]=1 nếu hub i phục vụ khách j
for i in range(data['n']):
    for j in range(data['m']):
        x[i,j] = solver.IntVar(0, 1, 'x[{}][{}]'.format(i, j))

# Ràng buộc
# 1: Khách nào cũng được phục vụ
for j in range(data['m']):
    solver.Add(sum(x[i,j] for i in data['nearestHub'][j]) == 1)

# 2: Số hàng của khách nằm trong khả năng phục vụ
for i in range(data['n']):
    solver.Add(sum(x[i,j]*data['vc'][j] for j in range(data['m'])) <= data['vh'][i])

# Hàm mục tiêu
solver.Minimize(sum([distance(i,j)*x[i,j] for i in range(data['n']) for j in range(data['m'])]))
# Solve
status=solver.Solve()
# Print solution

sol = np.zeros(shape=data['m'])
cost= -1
if status == pywraplp.Solver.FEASIBLE or status == pywraplp.Solver.OPTIMAL :
    cost = solver.Objective().Value()
    for i in range(data['n']):
        for j in range(data['m']):
            if (x[i,j].solution_value()==1):
                print(str(i)+" "+str(j))
                sol[j]=i+1
else:
    print("No solution")
print('time: ', time.time()-t)

0 408
0 446
0 2083
0 2130
0 2163
0 2461
0 2538
0 2629
0 2698
0 2988
0 3693
0 4491
1 252
1 281
1 1238
1 3400
1 3531
1 3844
1 4291
1 4654
2 333
3 97
3 573
3 898
3 1015
3 1161
3 1328
3 1405
3 1871
3 2739
3 3098
3 3283
3 3287
3 3317
3 3369
3 3455
3 3590
3 3650
3 4136
3 4848
4 729
4 926
4 997
4 1031
4 1109
4 1339
4 2271
4 2649
4 3050
4 3357
4 4734
5 92
5 1261
5 1388
5 1857
5 2961
5 3202
5 3263
5 3431
5 3799
5 4064
5 4352
5 4662
6 266
6 1335
6 1484
6 1561
6 1566
6 2345
6 2680
6 3016
6 3116
6 3438
6 3985
6 4083
6 4128
6 4323
7 62
7 375
7 1383
7 1684
7 1953
7 2106
7 3130
7 3179
7 3921
7 3989
7 4062
7 4437
7 4471
7 4623
7 4723
8 1165
8 2456
8 2735
8 2758
8 3813
8 3965
8 4075
8 4236
8 4679
9 230
9 588
9 2051
9 3326
9 3962
9 4351
10 1776
10 2052
10 2399
10 3228
11 29
11 1720
11 3870
11 4216
12 12
12 138
12 322
12 664
12 745
12 1220
12 1410
12 1568
12 1801
12 3463
12 3684
12 4625
13 1534
13 3871
14 841
14 844
14 1537
14 1866
14 3454
15 368
15 470
15 636
15 733
15 850
15 885
15 915
15 962
15 1211
1

129 400
129 984
129 1105
129 1136
129 1357
129 1499
129 1510
129 1677
129 2159
129 2320
129 2384
129 2459
129 2769
129 2810
129 2974
129 3313
129 3676
129 3853
129 3893
129 4338
129 4459
129 4838
130 132
130 215
130 814
130 819
130 1168
130 1736
130 1778
130 2224
130 2295
130 2782
130 2929
130 3730
130 3814
130 4185
130 4546
130 4828
130 4994
131 343
131 855
131 864
131 873
131 875
131 914
131 2561
131 2731
131 2885
131 3039
131 3345
131 3394
131 3466
131 3652
131 3785
131 4845
132 1008
132 1549
132 2636
132 3399
132 3555
132 3753
132 4448
133 218
133 371
133 1379
133 1500
133 1903
133 1992
133 2788
133 3052
133 4527
134 2234
135 2
135 1851
135 2665
135 2941
135 3205
135 3802
135 4447
135 4768
135 4826
137 362
137 1640
137 1800
137 2260
137 2432
137 2734
137 2890
137 3023
137 3473
137 3732
137 4116
137 4678
138 124
138 958
138 2066
138 2327
138 2338
138 3224
138 3439
138 4356
139 399
139 2344
139 2391
139 2980
139 3237
139 3646
139 3744
139 4131
139 4962
140 197
140 358
140 533
140 114

255 3253
255 3955
255 4211
255 4381
255 4543
255 4701
255 4758
256 971
256 1248
256 1705
256 1732
256 2147
256 2331
257 109
257 144
257 316
257 643
257 823
257 896
257 2796
257 2884
257 3056
257 3605
257 3641
257 3714
257 3907
257 4058
257 4362
258 72
258 271
258 1050
258 1688
258 1707
258 1867
258 2092
258 2683
258 3426
259 77
259 272
259 402
259 845
259 1645
259 2047
259 2893
259 3046
259 3087
259 3362
259 3386
259 3509
259 3752
259 4155
259 4284
259 4430
259 4644
260 74
260 75
260 86
260 569
260 1474
260 2198
260 2392
260 2679
260 3336
260 3376
260 3710
260 4126
260 4276
260 4487
261 33
261 662
261 1217
261 1496
261 1512
261 2112
261 2210
261 2357
261 2604
261 2646
261 3427
261 3721
261 3760
261 3798
261 3826
261 4169
261 4739
261 4753
261 4912
261 4976
262 401
262 907
262 1200
262 1374
262 2156
262 2293
262 2396
262 2444
262 3049
262 3229
262 3318
262 4494
263 722
263 1127
263 1142
263 2908
263 4292
264 4
264 1316
264 1562
264 2743
264 3168
264 3658
264 3839
264 4369
265 175
265 28

404 3569
404 3937
404 4087
404 4364
404 4408
404 4937
405 302
405 1547
405 2107
405 2564
405 3143
405 4397
406 711
406 1140
406 2912
406 3433
406 3492
407 183
407 704
407 1988
407 2014
407 3153
407 3232
407 3256
407 4055
407 4142
407 4234
407 4418
407 4660
408 603
408 1076
408 1571
408 3688
408 4454
409 1578
409 2149
409 3134
409 3406
409 3975
410 304
410 685
410 3081
410 4240
411 222
411 247
411 290
411 344
411 558
411 1139
411 1322
411 1552
411 1559
411 1573
411 1634
411 1962
411 2253
411 2319
411 2375
411 2383
411 3165
411 3183
411 3701
411 3719
411 3847
411 3984
411 4072
411 4153
411 4586
411 4618
411 4968
412 627
412 717
412 1163
412 1452
412 2296
412 3993
412 4431
412 4969
414 60
414 414
414 805
414 880
414 2252
414 2530
414 2531
415 100
415 288
415 601
415 2581
415 2928
415 2985
415 3852
415 4745
416 200
416 318
416 727
416 773
416 1000
416 1300
416 1961
416 2213
416 2302
416 3803
416 4135
416 4200
416 4640
416 4858
417 1170
417 1617
417 1699
417 1750
417 2881
417 3280
417 3645


In [5]:
f = open(f_o, "w")
# result = cost(sol)
if (cost <0):
    f.write("-1 \n")
else: 
    f.write(str(cost) +"\n")
    for i in range(data["m"]):
        f.write(str(sol[i])+" ")
f.close()

print(cost)
print(sol)

11779.655894210504
[157. 118. 136. ... 394. 144. 119.]
